<h2>Un tour au niveau des arbres de décision! Code self Decision tree algo. Don't use it for project's algorithms. it's for learn !!!</h2>
<h3>1.1- Importation des packages</h3>

In [1]:
import numpy as np
import pandas as pd
from functools import reduce
import operator

<h3>1.2 Deal with data. Here we choose to predict a price of a house based on surface, est_new and nb_piece. We will write a decision tree who deal with data</h3>

In [3]:
data = {
    "prix": [150000, 200000, 175000, 300000, 250000, 180000, 220000, 275000, 195000, 320000, 280000, 210000, 240000, 350000, 325000, 265000, 230000, 205000, 195000, 185000],
    "surface": [120.5, 145.0, 110.2, 200.0, 175.5, 130.0, 160.0, 190.2, 140.8, 210.5, 195.0, 150.2, 170.0, 225.5, 215.0, 180.5, 155.2, 135.0, 125.5, 115.0],
    "est_new": [True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False, True, False],
    "nb_piece": [3, 4, 3, 5, 4, 3, 4, 5, 3, 5, 4, 3, 4, 5, 4, 3, 4, 5, 3, 4]
}

data_frame = pd.DataFrame(data)
print(data_frame)

      prix  surface  est_new  nb_piece
0   150000    120.5     True         3
1   200000    145.0    False         4
2   175000    110.2     True         3
3   300000    200.0    False         5
4   250000    175.5     True         4
5   180000    130.0    False         3
6   220000    160.0     True         4
7   275000    190.2    False         5
8   195000    140.8     True         3
9   320000    210.5    False         5
10  280000    195.0     True         4
11  210000    150.2    False         3
12  240000    170.0     True         4
13  350000    225.5    False         5
14  325000    215.0     True         4
15  265000    180.5    False         3
16  230000    155.2     True         4
17  205000    135.0    False         5
18  195000    125.5     True         3
19  185000    115.0    False         4


In [ ]:
class DecisionTree:
    def __init__(self, target, max_depth, dataframe):
        self.target = target
        self.max_depth = max_depth
        self.dataframe = dataframe


    def __quanti_reduce__(self, feature, value, dataframe):
        """reduction de données quantitative basé sur le feature et value"""
        left = dataframe[dataframe.iloc[:, feature] <= value]
        right = dataframe[dataframe.iloc[: feature] > value ]

        return left, right


    def __quali_reduce__(self, feature, value, dataframe):
        """reduction de données qualitatives basé sur le feature et value"""
        left = dataframe[dataframe.iloc[:, feature] == value]
        right = dataframe[dataframe.iloc[: feature] != value ]

        return left, right

    def __mse__(self, dataset):
        """Evaluation du coût de prediction dune node """
        rows = dataset[self.target]
        pred = np.ones(rows.shape[0]) * np.mean(rows)
        result = 1/len(rows) * np.sum((rows-pred)**2)
        return result


    def __split_evalutor__(self, left_datasets, right_datasets):
        """cout de separation d'un node. Encore appelée cout d'une node.
        Ici on fait la somme de produit m1*Evc, m1 pour le rapport des echantillons
        et Evc pour l'evalution mse du coté (gauche ou droite)
        """
        nb_left = len(left_datasets)
        left_evc = self.__mse__(left_datasets)
        nb_right = len(left_datasets)
        right_evc = self.__mse__(right_datasets)

        nb_total = nb_right + nb_left

        evaluator = (nb_left * left_evc)/nb_total + (nb_right * right_evc)/nb_total

        return  evaluator

    def __test_quanti__(self, feature, dataframe):
        col = ("feature", "value", "nature", "cost")

        test_result = pd.DataFrame([], columns=col)
        value_test = (dataframe.loc[:, feature].sort_values()[1:].values +
                      dataframe.loc[:, feature].sort_values()[:-1].values)/2
        for value in value_test:
            left, right = self.__quanti_reduce__(feature, value, dataframe)
            cost = self.__split_evalutor__(left, right)
            test_result.append({"feature": feature, "value": value, "nature": "quanti", "cost": cost})

        return test_result

    def __test_quali__(self, feature, dataframe):
        col = ("feature", "value", "nature", "cost")

        test_result = pd.DataFrame([], columns=col)
        value_test = dataframe.loc[:, feature].unique().values
        for value in value_test:
            left, right = self.__quali_reduce__(feature, value, dataframe)
            cost = self.__split_evalutor__(left, right)
            test_result.append({"feature": feature, "value": value, "nature": "quali", "cost": cost})

        return test_result

    def __best_separtor(self, dataframe):
        """find the best separator for each column"""
        test = None
        col = ("feature", "value", "nature", "cost")
        best_result = pd.DataFrame([], columns=col)
        columns = dataframe.columns[np.logical_not(dataframe.columns == self.target)]
        for colum in columns:
            if len(dataframe[colum].unique()) >=5:
                test = self.__test_quanti__(colum, dataframe)
            elif len(dataframe[colum].unique() > 1):
                test = self.__test_quali__(colum, dataframe)
            best_result = best_result.append(test)

        best_result = best_result.reset_index(drop=True)
        idx_min_column = best_result["cost"].idxmin(axis=0,alpha=True)

        return best_result.iloc[idx_min_column,:1]

    def __create_leaf(self, dataframe):
        label = dataframe[self.target]
        pop = len(label.shape[0])
        prediction = np.means(label)

        return Leaf(label,pop, prediction)


    def __training__(self):

class Leaf:
    def __init__(self, label, pop, prediction):
        self.label = label
        self.pop = pop
        self.prediction = prediction






